###Importing libraries

In [1]:
import numpy as np
import pandas as pd

###Importing dataset (from google drive)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive
!ls

/content/drive/MyDrive
 c1_BoW_Sentiment_Model.pkl
 c2_Classifier_Sentiment_Model
'colab folder'
'Colab Notebooks'
'flutter devs.gsheet'
'IBC Hack Asset_Management_System_using_blockchain.pptx'
'pickl.ai Datathon'
'presentation script.gdoc'
 ResumeUpdated.pdf


In [4]:
dataset = pd.read_csv('Colab Notebooks/a2_RestaurantReviews_FreshDump.tsv', delimiter = '\t', quoting = 3)
dataset.head()

,Review
0,Spend your money elsewhere.
1,Their regular toasted bread was equally satisf...
2,The Buffet at Bellagio was far from what I ant...
3,"And the drinks are WEAK, people!"
4,-My order was not correct.


###Data cleaning

In [5]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
corpus=[]

for i in range(0, 100):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

###Data transformation

In [7]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='c1_BoW_Sentiment_Model.pkl'
# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open(cvFile, "rb"))


In [8]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(100, 1420)

###Predictions (via sentiment classifier)

In [9]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')

In [10]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[0 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 1
 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1 1 0 0 1 1 1
 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0]


In [11]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head()

,Review,predicted_label
0,Spend your money elsewhere.,0
1,Their regular toasted bread was equally satisf...,1
2,The Buffet at Bellagio was far from what I ant...,1
3,"And the drinks are WEAK, people!",1
4,-My order was not correct.,1


In [13]:
dataset.tail()

,Review,predicted_label
95,I think food should have flavor and texture an...,1
96,Appetite instantly gone.,0
97,Overall I was not impressed and would not go b...,0
98,"The whole experience was underwhelming, and I ...",0
99,"Then, as if I hadn't wasted enough of my life ...",0


In [12]:
dataset.to_csv("c3_Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)